Lotka Volterra problem

In [1]:
# We import the necessary libraries
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from deepxde.backend import tf

Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Since this is a trivial problem, we know the values of R and U a priori but here we will use them to solve
# the problem so that we can then check how well our NN performs when estimating them.
rb = 20 # real value of R
ub = 200 # real value of U

# Function to simulate system
def func(t, r):
    x, y = r
    dx_t = 1 / ub * rb * (2.0 * ub * x - 0.04 * ub * x * ub * y)
    dy_t = 1 / ub * rb * (0.02 * ub * x * ub * y - 1.06 * ub * y)
    return dx_t, dy_t

# Function that generates the true solution of the problem
def gen_truedata():
    t = np.linspace(0., 1., 25)

    sol = integrate.solve_ivp(func, (0, 1), (100 / ub, 15 / ub), t_eval=t)
    x_true, y_true = sol.y
    x_true = x_true.reshape(25, 1)
    y_true = y_true.reshape(25, 1)

    return t, np.hstack([x_true, y_true])


From now on, we "forget" the values of R and U and we will "find" them using our Neural Network. At the end
we will check how well it has performed.

In [3]:
# Our aim is to find the 2 unknown variables (R and U) for the system of equations.
#R = dde.Variable(15.) # R
U = dde.Variable(215.) # U

In [4]:
# Express the ODE system, x represents the coordinate t, y is a 2D vector that contains r(t) and p(t)
def LV_system(x, y):
    r = y[:, 0:1]
    p = y[:, 1:2]
    dr_t = dde.grad.jacobian(y, x, i=0)
    dp_t = dde.grad.jacobian(y, x, i=1)
    #return [
    #    dr_t - 1 / U * R * (2.0 * U * r - 0.04 * U * r * U * p),
    #    dp_t - 1 / U * R * (0.02 * r * U * p * U - 1.06 * p * U),
    #]
    return [
            dr_t - 1 / U * 20 * (2.0 * U * r - 0.04 * U * r * U * p),
            dp_t - 1 / U * 20 * (0.02 * r * U * p * U - 1.06 * p * U),
        ]
# x represents the t coordinate
# y represents the network output ie the solution y(r(t),p(t))

In [5]:
# We need to implement a function that returns true for points inside the subdomain and false for the points outside
def boundary(_, on_initial):
    return on_initial

In [6]:
# Define a time domain
geom = dde.geometry.TimeDomain(0.0, 1.0)

In [7]:
# We specify the initial conditions
#ic1 = dde.icbc.IC(geom, lambda X: 100/C2, boundary, component=0)
#ic2 = dde.icbc.IC(geom, lambda X: 15/C2, boundary, component=1)
ic1 = dde.icbc.IC(geom, lambda X: 100/ub, boundary, component=0)
ic2 = dde.icbc.IC(geom, lambda X: 15/ub, boundary, component=1)

In [8]:
# Organize and assign training data
observe_t, ob_y = gen_truedata()
observe_t = observe_t.reshape(25, 1)
observe_y0 = dde.icbc.PointSetBC(observe_t, ob_y[:, 0:1], component=0)
observe_y1 = dde.icbc.PointSetBC(observe_t, ob_y[:, 1:2], component=1)

In [9]:
# Define the ODE problem
data = dde.data.PDE(
    geom,
    LV_system,
    [ic1, ic2, observe_y0, observe_y1],
    num_domain=3000,
    num_boundary=2,
    anchors=observe_t) # extra points used for training, added reading lorenz

In [10]:
# Create the net
layer_size = [1] + [64] * 6 + [2]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

In [11]:
# We add a feature layer with sin(kt) since we expect Lotka-Volterra to be periodic. The sin(kt) forces the prediction
# to be periodic and thus more accurate
def input_transform(t):
    return tf.concat(
        (
            t,
            tf.sin(t),
            tf.sin(2 * t),
            tf.sin(3 * t),
            tf.sin(4 * t),
            tf.sin(5 * t),
            tf.sin(6 * t),
        ),
        axis=1,
    )

In [12]:
def output_transform(t, y):
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]

    return tf.concat(
        [y1 * tf.tanh(t) + 100 / ub, y2 * tf.tanh(t) + 15 / ub], axis=1
    )

In [13]:
# We give indications that C1 and C2 need to be trained alongside the other parameters in the NN. This
# is how we estimate their value. They will then be saved in a file called variables.dat
external_trainable_variables = [U]
#external_trainable_variables = [R, U]
variable = dde.callbacks.VariableValue(
    external_trainable_variables, period=600, filename="variables.dat"
)

In [14]:
# We add the layer to impose periodicity we've just created
net.apply_feature_transform(input_transform)
net.apply_output_transform(output_transform)

In [15]:
# Now that the NN has been defined, we build a model, set the optimizer and learning rate and train it for 50,000 iterations
#model = dde.Model(data, net)
#model.compile("adam", lr=0.001)
#losshistory, train_state = model.train(iterations=50000)
model = dde.Model(data, net)
model.compile(
    "adam", lr=0.001, external_trainable_variables=external_trainable_variables
)
losshistory, train_state = model.train(iterations=60000, callbacks=[variable])

Compiling model...
Building feed-forward neural network...
'build' took 0.030904 s



/Users/giuliadesanctis/PycharmProjects/compStat/venv/lib/python3.11/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:116: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 0.339932 s

Training model...

Step      Train loss                                                      Test loss                                                       Test metric
0         [6.34e+02, 4.35e+00, 0.00e+00, 0.00e+00, 1.46e-01, 1.47e-01]    [6.34e+02, 4.35e+00, 0.00e+00, 0.00e+00, 1.46e-01, 1.47e-01]    []  


2024-03-10 20:18:20.653260: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


1000      [2.74e+00, 7.17e-01, 0.00e+00, 0.00e+00, 9.82e-02, 3.93e-02]    [2.74e+00, 7.17e-01, 0.00e+00, 0.00e+00, 9.82e-02, 3.93e-02]    []  
2000      [1.61e+00, 6.65e-01, 0.00e+00, 0.00e+00, 9.17e-02, 3.46e-02]    [1.61e+00, 6.65e-01, 0.00e+00, 0.00e+00, 9.17e-02, 3.46e-02]    []  
3000      [1.46e+01, 7.57e-01, 0.00e+00, 0.00e+00, 1.03e-01, 4.39e-02]    [1.46e+01, 7.57e-01, 0.00e+00, 0.00e+00, 1.03e-01, 4.39e-02]    []  
4000      [5.71e+00, 8.78e-01, 0.00e+00, 0.00e+00, 9.82e-02, 3.87e-02]    [5.71e+00, 8.78e-01, 0.00e+00, 0.00e+00, 9.82e-02, 3.87e-02]    []  
5000      [3.38e+00, 9.03e-01, 0.00e+00, 0.00e+00, 9.58e-02, 3.77e-02]    [3.38e+00, 9.03e-01, 0.00e+00, 0.00e+00, 9.58e-02, 3.77e-02]    []  
6000      [2.54e+00, 7.90e-01, 0.00e+00, 0.00e+00, 9.24e-02, 3.60e-02]    [2.54e+00, 7.90e-01, 0.00e+00, 0.00e+00, 9.24e-02, 3.60e-02]    []  
7000      [2.05e+00, 7.14e-01, 0.00e+00, 0.00e+00, 8.77e-02, 3.26e-02]    [2.05e+00, 7.14e-01, 0.00e+00, 0.00e+00, 8.77e-02, 3.26e-02]    []  

In [16]:
# To further minimize the loss, we continue with L-BFGS
model.compile("L-BFGS")
losshistory, train_state = model.train()
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

Compiling model...
'compile' took 0.139420 s

Training model...

Step      Train loss                                                      Test loss                                                       Test metric
60000     [3.93e-04, 2.40e-04, 0.00e+00, 0.00e+00, 1.44e-03, 6.83e-04]    [3.93e-04, 2.40e-04, 0.00e+00, 0.00e+00, 1.44e-03, 6.83e-04]    []  
61000     [1.94e-04, 1.53e-04, 0.00e+00, 0.00e+00, 1.22e-03, 5.92e-04]    [1.94e-04, 1.53e-04, 0.00e+00, 0.00e+00, 1.22e-03, 5.92e-04]        
62000     [1.93e-04, 1.55e-04, 0.00e+00, 0.00e+00, 1.17e-03, 5.67e-04]    [1.93e-04, 1.55e-04, 0.00e+00, 0.00e+00, 1.17e-03, 5.67e-04]        
63000     [1.76e-04, 1.49e-04, 0.00e+00, 0.00e+00, 1.05e-03, 5.09e-04]    [1.76e-04, 1.49e-04, 0.00e+00, 0.00e+00, 1.05e-03, 5.09e-04]        
64000     [1.79e-04, 1.38e-04, 0.00e+00, 0.00e+00, 8.56e-04, 4.22e-04]    [1.79e-04, 1.38e-04, 0.00e+00, 0.00e+00, 8.56e-04, 4.22e-04]        
65000     [1.45e-04, 1.21e-04, 0.00e+00, 0.00e+00, 6.80e-04, 3.36e-04]

KeyboardInterrupt: 

In [ ]:
plt.xlabel("t")
plt.ylabel("population")

t, y_true = gen_truedata()
plt.plot(t, y_true[:, 0], color="black", label="x_true")
plt.plot(t, y_true[:, 1], color="blue", label="y_true")

In [ ]:
t = t.reshape(100, 1)
sol_pred = model.predict(t)
x_pred = sol_pred[:, 0:1]
y_pred = sol_pred[:, 1:2]

plt.plot(t, x_pred, color="red", linestyle="dashed", label="x_pred")
plt.plot(t, y_pred, color="orange", linestyle="dashed", label="y_pred")
plt.legend()
plt.show()

In [ ]:
filename = '/Users/giuliadesanctis/PycharmProjects/compStat/lotka_volterra/variables.dat'
with open(filename) as file:
    lines = [line.rstrip() for line in file]

In [ ]:
C1 = []
C2 = []
C3 = []
for line in lines:
    line = line.split(" ")
    C1.append(float(line[1].replace(",", "").replace("[","").replace("]","")))
C1

In [ ]:
len(C1)
xx = np.linspace(0,60000,101)

plt.xlabel("Num. iterations")
plt.ylabel("Prediction of U")
plt.title ("Prediction of U - R frozen - initialized at 215")
plt.scatter(xx,C1)
plt. axhline(y=200, color='r', linestyle='--', linewidth=2)